<div align="center">

# Работа с категориальными признаками

</div>

---

Категориальные данные бывают двух видов:

* **Порядковые** — категории можно упорядочить по смыслу. Например, размеры одежды: маленький < средний < большой. Здесь есть естественный порядок.
* **Номинальные** — категории просто разные, без порядка. Например, цвета: красный, синий, зелёный — нельзя сказать, что один цвет «больше» другого.

То есть, порядковые данные можно сравнивать и упорядочивать, а номинальные — нет.


In [115]:
# Создание  DataFrame с демонстрационными данными
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [116]:
df = pd.DataFrame([
            ['green', 'M', 10.1, 'class2'],
            ['red', 'L', 13.5, 'class1'],
            ['blue', 'XL', 15.3, 'class2']])
df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


В этом DataFrame есть три типа признаков: **номинальный (цвет)**, **порядковый (размер)** и **числовой (цена)**. Метки классов — последний столбец.

Чтобы алгоритм правильно понял порядок категориальных признаков (например, размер одежды), нужно вручную заменить текстовые метки числами, отражающими их порядок. Автоматически это сделать сложно, поэтому задаём соответствие сами, например:
M = 1, L = 2, XL = 3.

In [117]:
size_mapping = {'XL': 3,
                'L': 2,
                'M': 1}
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


Если нужно вернуть числа обратно в исходные строки, достаточно создать обратный словарь с соответствиями и применить его к столбцу через `map` в pandas. Это легко преобразует числовые значения обратно в текстовые метки.

In [118]:
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

## Кодирование меток класса

Многие библиотеки требуют, чтобы метки классов были числами. Хотя scikit-learn обычно сам это делает, лучше заранее представить метки как целые числа, чтобы избежать ошибок. Для этого можно просто сопоставить каждому классу уникальное число без учёта порядка — например, начать с 0 и далее по порядку.

Затем использовать словарь сопоставлений для преобразования меток класса в целые числа.

In [119]:
class_mapping = {label: idx for idx, label in
                 enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [120]:
# Преобразование меток класса в целые числа
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


Чтобы вернуть числовые метки классов обратно в исходные строки, достаточно создать обратный словарь, поменяв местами ключи и значения, и применить его для преобразования чисел обратно в метки.

In [121]:
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


Альтернативно можно использовать `LabelEncoder` из scikit-learn — удобный класс, который автоматически преобразует строковые метки классов в целые числа и обратно.

In [122]:
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([1, 0, 1])

Метод `fit_transform` сочетает в себе вызовы `fit` и `transform` — сначала обучается на данных, затем сразу преобразует их. Чтобы вернуть числовые метки обратно в строки, используют метод `inverse_transform`.

In [123]:
class_le.inverse_transform(y)

array(['class2', 'class1', 'class2'], dtype=object)

## Позиционное кодирование номинальных признаков

Поскольку метки классов в scikit-learn считаются номинальными (без порядка), для их кодирования удобно использовать `LabelEncoder`. 
Аналогично, этим же методом можно преобразовать любой номинальный признак, например, столбец с цветами.


In [124]:
X = df[['color', 'size', 'price']].values
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

Если просто заменить категории числовыми метками (например, цвет "blue" — 0, "green" — 1, "red" — 2), то многие модели машинного обучения будут считать, что между этими числами есть порядок и расстояния. Например, модель может понять, что "green" больше "blue", а "red" больше "green". Но для номинальных признаков, таких как цвет, такой порядок не имеет смысла — "red" не «больше» и не «меньше» "green".

Из-за этого модель может неправильно интерпретировать данные и делать менее точные предсказания.

Чтобы избежать этой ошибки, применяют **one-hot кодирование** (или прямое позиционное кодирование). Его идея в том, что для каждого уникального значения признака создаётся отдельный бинарный столбец (фиктивный признак). В нашем примере для цвета создаются три столбца — "blue", "green" и "red". В каждой строке будет 1 в столбце, соответствующем цвету этого образца, и 0 в остальных.

Например:

* "green" → \[0, 1, 0]
* "red" → \[0, 0, 1]
* "blue" → \[1, 0, 0]

Такой способ кодирования убирает искусственный порядок и позволяет модели правильно работать с номинальными признаками.

Для этого в scikit-learn есть удобный класс `OneHotEncoder` из модуля `preprocessing`, который автоматически выполняет это преобразование.



In [125]:
X = df[['color', 'size', 'price']].values
color_ohe = OneHotEncoder()
color_ohe.fit_transform(X[:, 0].reshape(-1, 1)).toarray()


array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

Важно отметить, что мы применили `OneHotEncoder` только к одному столбцу (например, `X[:, 0].reshape(-1, 1)`), чтобы случайно не преобразовать остальные признаки. Если у вас есть массив с несколькими признаками, и вы хотите кодировать **только определённые столбцы**, удобно использовать **`ColumnTransformer`** из scikit-learn.

`ColumnTransformer` позволяет применять разные преобразования к разным столбцам. Он принимает список кортежей вида:

```
(name, transformer, columns)
```

Где:

* `name` — произвольное имя преобразования,
* `transformer` — что применять (например, `OneHotEncoder()`),
* `columns` — какие столбцы трансформировать (номер или список).

Это особенно полезно, когда у вас смесь числовых, порядковых и категориальных данных, и каждому типу требуется своё преобразование.


In [126]:
X = df[['color', 'size', 'price']].values
c_transf = ColumnTransformer([
    ('onehot', OneHotEncoder(), [0]),
    ('nothing', 'passthrough', [1, 2])
])
c_transf.fit_transform(X).astype(float)

array([[ 0. ,  1. ,  0. ,  1. , 10.1],
       [ 0. ,  0. ,  1. ,  2. , 13.5],
       [ 1. ,  0. ,  0. ,  3. , 15.3]])

В приведённом примере с `ColumnTransformer` мы использовали аргумент `'passthrough'`, чтобы изменить только указанный столбец (например, с цветом), а остальные признаки (например, размер и цена) оставить без изменений. Это удобно, когда нужно выборочно применять преобразования.

Ещё более простой способ создать фиктивные (one-hot) признаки — использовать метод `pandas.get_dummies()`. Он автоматически находит все **строковые столбцы** в `DataFrame` и кодирует их, при этом оставляя **числовые столбцы без изменений**. Это быстрый и удобный способ для позиционного кодирования категориальных признаков прямо в `pandas`, без необходимости вручную указывать столбцы.


Удаление одного из столбцов при one-hot кодировании (через `drop='first'`) нужно, чтобы избежать **мультиколлинеарности** — ситуации, когда один признак можно точно выразить через другие. Это особенно важно для линейных моделей (например, логистической регрессии), где избыточные признаки могут мешать обучению.

Параметр `drop='first'` в `OneHotEncoder` удаляет первый столбец из каждой категории, сохраняя при этом всю информацию, но без избыточности.
Параметр `categories='auto'` говорит, что уникальные значения категорий следует определять автоматически — это поведение по умолчанию.

In [127]:
color_ohe = OneHotEncoder(categories = 'auto', drop = 'first')
c_transf = ColumnTransformer([
    ('onehot', color_ohe, [0]),
    ('nothing', 'passthrough', [1, 2])
])
c_transf.fit_transform(X).astype(float)

array([[ 1. ,  0. ,  1. , 10.1],
       [ 0. ,  1. ,  2. , 13.5],
       [ 0. ,  0. ,  3. , 15.3]])

Хотя one-hot кодирование — самый распространённый способ обработки категориальных признаков, при большом числе уникальных значений оно может создавать слишком много столбцов. В таких случаях есть альтернативы:

* **Бинарное кодирование**: категории преобразуются в числа, затем в двоичный формат — это снижает число признаков до примерно `log2(K)`, где K — количество категорий.
* **Частотное и счётное кодирование**: категории заменяются на частоту или количество их появлений в данных.

Эти методы доступны в библиотеке `category_encoders`, совместимой с scikit-learn. Хотя они не всегда лучше one-hot, их можно использовать как **гиперпараметр**, чтобы улучшить модель в некоторых задачах.
